# ED-NHANES CDSS (HTN + HF)
## Agentic XAI Clinical Decision Support Demo

This Colab notebook runs the full reproducible pipeline:

1. Install dependencies  
2. Download NHANES data  
3. Build merged dataset  
4. Train leakage-free pretest models  
5. Run a single-patient CDSS report  
6. Export SHAP figures for the paper  

Repo: https://github.com/003733m/ed-cdss-agentic-xai


In [ ]:
!git clone https://github.com/003733m/ed-cdss-agentic-xai.git
%cd ed-cdss-agentic-xai
!pip install -r requirements.txt
import os, urllib.request

os.makedirs("data", exist_ok=True)

urls = {
    "DEMO_J.XPT": "https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2017/DataFiles/DEMO_J.xpt",
    "BPX_J.XPT":  "https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2017/DataFiles/BPX_J.xpt",
    "BPQ_J.XPT":  "https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2017/DataFiles/BPQ_J.xpt",
    "BMX_J.XPT":  "https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2017/DataFiles/BMX_J.xpt",
    "SMQ_J.XPT":  "https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2017/DataFiles/SMQ_J.xpt",
    "MCQ_J.XPT":  "https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2017/DataFiles/MCQ_J.xpt",
    "DIQ_J.XPT":  "https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2017/DataFiles/DIQ_J.xpt",
    "ALQ_J.XPT":  "https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2017/DataFiles/ALQ_J.xpt",
    "PAQ_J.XPT":  "https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2017/DataFiles/PAQ_J.xpt",
    "RXQ_RX_J.XPT": "https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2017/DataFiles/RXQ_RX_J.xpt",
}

for f, u in urls.items():
    p = f"data/{f}"
    if not os.path.exists(p):
        print("Downloading", f)
        urllib.request.urlretrieve(u, p)

print("NHANES files ready.")


In [ ]:
!python -m scripts.build_dataset --data-folder data --out-csv data/NHANES_Merged_Data.csv
!python -m scripts.train_htn_pretest --csv data/NHANES_Merged_Data.csv --artifacts-dir artifacts
!python -m scripts.train_hf_pretest  --csv data/NHANES_Merged_Data.csv --artifacts-dir artifacts
import os
os.environ["LANGSMITH_API_KEY"] = "lsv2_XXXXXXXX"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGSMITH_PROJECT"] = "ed-cdss-paper-demo"


In [ ]:
!python -m scripts.run_demo --csv data/NHANES_Merged_Data.csv --artifacts-dir artifacts --target-idx 14174
!python -m scripts.export_figures --csv data/NHANES_Merged_Data.csv --out docs/figures
from IPython.display import Image, display

display(Image("docs/figures/htn_shap_summary.png"))
display(Image("docs/figures/hf_shap_summary.png"))
